# Omidyar Extractives Project 1
## OCR Contracts Backlog (Notebook 3 of 8)
### From directory of PDFs, performs OCR and outputs txt files to same directory

In [1]:
import io, os
import json
import unicodedata
from PIL import Image as PI
import pyocr
import pyocr.builders
from wand.image import Image
from tqdm import tqdm

In [2]:
# Where you want to save the PDFspip 
destination_folder = 'contract_data/Contracts_Backlog/'

In [4]:
pdfs = [unicodedata.normalize('NFKC',f.decode('utf8')) for f in os.listdir(destination_folder) if f.lower().endswith('.pdf')]
txt_files = [unicodedata.normalize('NFKC',f.decode('utf8')) for f in os.listdir(destination_folder) if f.lower().endswith('.txt')]
print len(pdfs)
print len(txt_files)

474
32


In [10]:
tempdir = "/private/var/tmp/"

### Perform OCR on PDFs

In [11]:
def ocr_pdf_to_text(filename):
    tool = pyocr.get_available_tools()[0]
    lang = 'spa'
    req_image = []
    final_text = []
    image_pdf = Image(filename=filename, resolution=300)
    image_jpeg = image_pdf.convert('jpeg')
    for img in image_jpeg.sequence:
        img_page = Image(image=img)
        req_image.append(img_page.make_blob('jpeg'))

    for img in req_image: 
        txt = tool.image_to_string(
            PI.open(io.BytesIO(img)),
            lang=lang,
            builder=pyocr.builders.TextBuilder()
        )
        final_text.append(txt)
    return final_text

In [12]:
errors = []
for filename in tqdm(pdfs):
    txt_file = filename[:-3] +'txt'
    txt_filename = destination_folder + txt_file
    
    if not txt_file in txt_files: 
        try:
            ocr_txt = ocr_pdf_to_text(destination_folder + filename)
            with open(txt_filename,'w') as f:
                for i in range(len(ocr_txt)):
                    f.write(json.dumps({i:ocr_txt[i].encode('utf8')}))
                    f.write('\n')
            f.close()

        except:
            print "Could not OCR " + filename

        files = os.listdir(tempdir)
        for file in files:
            if "magick" in file:
                os.remove(os.path.join(tempdir,file)) 

  6%|▋         | 30/474 [44:58<36:57:14, 299.63s/it]Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <

Could not OCR CTO_EyP_CACHICAMO__2006.pdf
Could not OCR CTO_EyP_CAG_6_2011.pdf
Could not OCR CTO_EyP_CAMPOS_TELLO_Y_LA_JAGUA__2007.pdf
Could not OCR CTO_EyP_CANAGUARO__2006.pdf
Could not OCR CTO_EyP_CAÑO_LOS_TOTUMOS_2010.pdf
Could not OCR CTO_EyP_CAÑO_SUR_2005.pdf
Could not OCR CTO_EyP_CAPORAL_2007.pdf
Could not OCR CTO_EyP_CARACOLI_2005.pdf
Could not OCR CTO_EyP_CARBONERA_2005.pdf
Could not OCR CTO_EyP_CARBONERA_2005_-_copia.pdf
Could not OCR CTO_EyP_CARDON_2012.pdf
Could not OCR CTO_EyP_CASANARE_ESTE_2005.pdf
Could not OCR CTO_EyP_CASCABEL__2006.pdf
Could not OCR CTO_EyP_CASIMENA__2005.pdf
Could not OCR CTO_EyP_CASTOR__2006.pdf
Could not OCR CTO_EyP_CAT_3_2012.pdf
Could not OCR CTO_EyP_CATGUAS_2005.pdf
Could not OCR CTO_EyP_CEBUCÁN__2007.pdf
Could not OCR CTO_EyP_CEDRELA__2010.pdf
Could not OCR CTO_EyP_CEIBA_2010.pdf
Could not OCR CTO_EyP_CERRERO__2007.pdf
Could not OCR CTO_EyP_CHAZA_2005.pdf
Could not OCR CTO_EyP_CHICUACO__2006.pdf
Could not OCR CTO_EyP_CHIGÜIRO_OESTE__2007.pdf
Coul

Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (em

 26%|██▌       | 123/474 [44:58<10:01:13, 102.77s/it]Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in 

Could not OCR CTO_EyP_DURILLO__2009.pdf
Could not OCR CTO_EyP_EL_EDÉN__2007.pdf
Could not OCR CTO_EyP_EL_PORTÓN__2007.pdf
Could not OCR CTO_EyP_EL_REMANSO__2007.pdf
Could not OCR CTO_EyP_EL_SANCY__2008.pdf
Could not OCR CTO_EyP_EL_TRIUNFO_2004.pdf
Could not OCR CTO_EyP_ESPERANZA_2004.pdf
Could not OCR CTO_EyP_FÉNIX__2007.pdf
Could not OCR CTO_EyP_FUERTE_NORTE__2006.pdf
Could not OCR CTO_EyP_FUERTE_SUR__2006.pdf
Could not OCR CTO_EyP_GABÁN__2006.pdf
Could not OCR CTO_EyP_GAITA_2007.pdf
Could not OCR CTO_EyP_GARAGOA__2011.pdf
Could not OCR CTO_EyP_GARIBAY__2005.pdf
Could not OCR CTO_EyP_GUA_2__2012.pdf
Could not OCR CTO_EyP_GUA_OFF_2_2012.pdf
Could not OCR CTO_EyP_GUACHIRIA_NORTE_2004.pdf
Could not OCR CTO_EyP_GUACHIRIA_SUR_2005.pdf
Could not OCR CTO_EyP_GUADUAL__2007.pdf
Could not OCR CTO_EyP_GUAMA__2007.pdf
Could not OCR CTO_EyP_GUARROJO__2006.pdf
Could not OCR CTO_EyP_GUASIMO_2004.pdf
Could not OCR CTO_EyP_GUATIQUIA_2007.pdf
Could not OCR CTO_EyP_HELEN__2006.pdf
Could not OCR CTO_EyP_

...

<bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
Exception TypeError: TypeError("object of type 'NoneType' has 

Could not OCR CTO_TEA_ARPA_-2005.pdf
Could not OCR CTO_TEA_BERILO__2008.pdf
Could not OCR CTO_TEA_BERRÍO_2009.pdf
Could not OCR CTO_TEA_BOGOTA__2008.pdf
Could not OCR CTO_TEA_BOROJÓ_-2005.pdf
Could not OCR CTO_TEA_CACHAMA_-2005.pdf
Could not OCR CTO_TEA_CAG_5-2011.pdf
Could not OCR CTO_TEA_CAIMAN_-2005.pdf
Could not OCR CTO_TEA_CANDILEJAS__2006.pdf
Could not OCR CTO_TEA_CARIOCA__2006.pdf
Could not OCR CTO_TEA_CAUCA_6_2011.pdf
Could not OCR CTO_TEA_CAUCA_7_2011.pdf
Could not OCR CTO_TEA_CHICAGO_-2005.pdf
Could not OCR CTO_TEA_CHIGÜIRO_-2005.pdf
Could not OCR CTO_TEA_COL_1_2014.pdf
Could not OCR CTO_TEA_COL_2_2012.pdf
Could not OCR CTO_TEA_COL_3_2012.pdf
Could not OCR CTO_TEA_COL_4_2014.pdf
Could not OCR CTO_TEA_COL_5_2012.pdf
Could not OCR CTO_TEA_COL_6_2014.pdf
Could not OCR CTO_TEA_COL_7_2014.pdf
Could not OCR CTO_TEA_COMADRE_NORTE_2005.pdf
Could not OCR CTO_TEA_COMUNEROS__2007.pdf
Could not OCR CTO_TEA_COR_15_2011.pdf
Could not OCR CTO_TEA_COR_46_2012.pdf
Could not OCR CTO_TEA_COR__2

Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (em

Could not OCR CTO_TEA_EL_CAUCHO_-2005.pdf
Could not OCR CTO_TEA_EL_CONCHAL_-2005.pdf
Could not OCR CTO_TEA_EL_QUESO_NORTE_-2005.pdf
Could not OCR CTO_TEA_EL_TIGRE_-2005.pdf
Could not OCR CTO_TEA_FOMEQUE_2009.pdf
Could not OCR CTO_TEA_FUERTE_2005.pdf
Could not OCR CTO_TEA_GUA_OFF_1_2012.pdf
Could not OCR CTO_TEA_GUA_OFF_3_2011.pdf
Could not OCR CTO_TEA_GUAICARAMO__2006.pdf
Could not OCR CTO_TEA_GUAIMARAL_2004.pdf
Could not OCR CTO_TEA_GUATIQUÍA__2006.pdf
Could not OCR CTO_TEA_GUEPARDO_-2005.pdf
Could not OCR CTO_TEA_HUMADEA_-2005.pdf
Could not OCR CTO_TEA_JAGÜEYES_-2005.pdf
Could not OCR CTO_TEA_LA_INDIA__2006.pdf
Could not OCR CTO_TEA_LA_MONA__2007.pdf
Could not OCR CTO_TEA_LA_UNIÓN__2006.pdf
Could not OCR CTO_TEA_LAS_BRISAS_-2005.pdf
Could not OCR CTO_TEA_LIGIA_2004.pdf
Could not OCR CTO_TEA_LOS_PICHACHOS_2009.pdf
Could not OCR CTO_TEA_LOWER_VILLETA__2009.pdf
Could not OCR CTO_TEA_MACAYA_2010.pdf
Could not OCR CTO_TEA_MANTECAL_2004.pdf
Could not OCR CTO_TEA_MARE_MARE_-2005.pdf
Could n


Exception TypeError: TypeError("object of type 'NoneType' has no len()",) in <bound method Image.__del__ of <wand.image.Image: (empty)>> ignored
